<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Product_Marketing_AI_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Marketing AI System

## Overview
This system helps create high-quality marketing images automatically. It takes in photos and optional audio or video, then processes, refines, and enhances them to produce beautiful marketing visuals for many different industries.

## Key Features

- **Easy Input:** Upload main and supplementary images, plus optional multimedia for extra context.
- **Smart Processing:** The system automatically cuts out key parts, improves image details, and boosts overall clarity.
- **Creative Prompts:** Custom prompts are generated to guide the image creation process, making it tailored to your needs.
- **Fast Generation:** Uses multiple AI models working together to generate and improve images quickly.
- **Quality Check:** Compares final images to the originals and provides simple quality feedback.
- **Simple Reports:** Automatically produces a brief report with the final prompt and quality scores.

## Benefits
- Saves time by automating the creation of professional marketing images.
- Provides consistent and attractive visuals optimized for your business.
- Easy to use with straightforward input and clear feedback.

Enjoy a seamless experience in making your marketing visuals stand out!

In [1]:
# Step 1: Install dependencies (for Colab Notebook; not needed if pre-installed)
%pip install --q git+https://github.com/facebookresearch/segment-anything.git
%pip install -q diffusers transformers accelerate
%pip install -q git+https://github.com/ChaoningZhang/Real-ESRGAN.git
%pip install -q git+https://github.com/advimman/lama.git
%pip install -q langchain langchain-google-genai groq openai u2net langchain_community
%pip install -q scikit-image
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O sam_vit_h.pth -q

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/Real-ESRGAN.git /tmp/pip-req-build-3pjgdb1p did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/Real-ESRGAN.git /tmp/pip-req-build-3pjgdb1p did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
ERROR: git+https://github.com/advimman/lama.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
ERROR: Could not find a version that satisfies the requirement u2net (from versions: none)
ERROR: No matching distribution found for u2net


In [4]:
# --- Step 2: Import Libraries ---
import os
import math
import hashlib
import numpy as np
import cv2
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from transformers import pipeline

# --- Step 3: Utility Functions ---
def free_memory(model):
    """Frees up GPU memory after use."""
    try:
        del model
        import torch
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"Memory cleanup error: {e}")

def verify_checksum(file_path, expected_hash):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        sha256.update(f.read())
    return sha256.hexdigest() == expected_hash

def psnr(target, ref):
    """Compute Peak Signal-to-Noise Ratio."""
    mse = np.mean((target - ref) ** 2)
    return 100 if mse == 0 else 20 * math.log10(255.0 / math.sqrt(mse))

# --- Step 4: Initialize Environment ---
print("Starting Product Marketing AI Pipeline...")
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Step 5: Load Models ---
print("Loading models...")

# Load SAM
from segment_anything import SamPredictor, sam_model_registry
sam_ckpt = "sam_vit_h.pth"
expected_hash = "your_real_checksum_here"  # Replace with actual SHA256
if not verify_checksum(sam_ckpt, expected_hash):
    raise ValueError("SAM checkpoint corrupted")
sam = sam_model_registry["vit_h"](checkpoint=sam_ckpt)
predictor = SamPredictor(sam)

# Load Stable Diffusion with ControlNet
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_seg", torch_dtype=torch.float16
)
sd_pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
).to("cuda")

# Load Real-ESRGAN
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

model = RRDBNet(num_in_ch=3, num_out_ch=3, nf=64, nb=23,
                gc=32, sf=4, norm_type='batch', act_type='leakyrelu',
                mode='CNA', convtype='Conv2D')
upscaler = RealESRGANer(scale=4, model_path='RealESRGAN_x4plus.pth', model=model, tile=0)

# --- Step 6: LLMs and Prompt Generation ---
from groq import Groq
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

from langchain_google_genai import ChatGoogleGenerativeAI
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

from openai import OpenAI
xai_client = OpenAI(base_url="https://api.x.ai/v1", api_key=os.environ.get("XAI_API_KEY"))

# HuggingFace LLM
text_generator = pipeline("text-generation", model="gpt2", device=0)
llm = HuggingFacePipeline(pipeline=text_generator)

print("✅ All models initialized successfully.")

Starting Product Marketing AI Pipeline...
Loading models...


ValueError: SAM checkpoint corrupted

In [5]:
# Define prompt templates
from langchain.prompts import PromptTemplate
prompt_templates = {
    "clothing": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="A model wearing {product} in {setting}, {style} aesthetic, {lighting} lighting, with {context}, {caption}, {external}."
    ),
    "food": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} on a table in {setting}, {style} ambiance, {lighting} lighting, delicious and appetizing, with {context}, {caption}, {external}."
    ),
    "electronics": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} in {setting}, {style} tech vibe, {lighting} lighting, sleek and modern, with {context}, {caption}, {external}."
    ),
    "hotels": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} in {setting}, {style} luxury ambiance, {lighting} lighting, inviting and upscale, with {context}, {caption}, {external}."
    ),
    "cosmetics": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} in {setting}, {style} glamorous setup, {lighting} lighting, vibrant and luxurious, with {context}, {caption}, {external}."
    ),
    "furniture": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} in {setting}, {style} cozy setting, {lighting} lighting, warm and inviting, with {context}, {caption}, {external}."
    ),
    "automotive": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} on {setting}, {style} adventurous vibe, {lighting} lighting, dynamic and bold, with {context}, {caption}, {external}."
    ),
    "travel": PromptTemplate(
        input_variables=["product", "setting", "style", "lighting", "context", "caption", "external"],
        template="{product} in {setting}, {style} vacation mood, {lighting} lighting, scenic and aspirational, with {context}, {caption}, {external}."
    )
}

In [6]:
# Step 4: Input Collection
def collect_inputs():
    try:
        print("Upload base image (e.g., model, table, room):")
        from google.colab import files
        base_upload = files.upload()
        base_image_path = list(base_upload.keys())[0]
        base_image = cv2.imread(base_image_path)
        if base_image is None:
            raise ValueError("Failed to load base image")
        base_image_rgb = cv2.cvtColor(base_image, cv2.COLOR_BGR2RGB)

        print("Upload secondary image (e.g., clothing, product, food):")
        secondary_upload = files.upload()
        secondary_image_path = list(secondary_upload.keys())[0]
        secondary_image = cv2.imread(secondary_image_path)
        if secondary_image is None:
            raise ValueError("Failed to load secondary image")
        secondary_image_rgb = cv2.cvtColor(secondary_image, cv2.COLOR_BGR2RGB)

        multimedia = None
        if input("Upload audio/video for context (y/n)? ").lower() == 'y':
            print("Upload audio/video file:")
            media_upload = files.upload()
            media_path = list(media_upload.keys())[0]
            multimedia = {'path': media_path, 'type': 'audio' if media_path.endswith(('.mp3', '.wav')) else 'video'}

        domain = input("Enter domain (e.g., clothing, food, electronics, hotels, cosmetics, furniture, automotive, travel): ")
        initial_prompt = input("Enter initial prompt (e.g., 'Model in a jacket, urban rooftop, edgy vibe'): ")

        return base_image_rgb, secondary_image_rgb, multimedia, domain.lower(), initial_prompt
    except Exception as e:
        print(f"Error collecting inputs: {e}")
        return None, None, None, None, None

In [7]:
# Step 5: Preprocess Images
def preprocess_images(base_image, secondary_image):
    try:
        predictor.set_image(base_image)
        h, w = base_image.shape[:2]
        input_point = np.array([[w // 2, h // 2]])
        input_label = np.array([1])
        masks, scores, _ = predictor.predict(point_coords=input_point, point_labels=input_label)
        mask = masks[np.argmax(scores)]
        mask_path = os.path.join(OUTPUT_DIR, "mask.png")
        cv2.imwrite(mask_path, (mask * 255).astype(np.uint8))

        if u2net:
            secondary_mask = u2net.predict(secondary_image)
            secondary_extracted = cv2.bitwise_and(secondary_image, secondary_image, mask=secondary_mask)
        else:
            gray = cv2.cvtColor(secondary_image, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
            secondary_extracted = cv2.bitwise_and(secondary_image, secondary_image, mask=thresh)
        secondary_path = os.path.join(OUTPUT_DIR, "secondary_extracted.png")
        cv2.imwrite(secondary_path, secondary_extracted)

        def normalize(image, size=(512, 512)):
            image = cv2.resize(image, size)
            image = cv2.convertScaleAbs(image, alpha=1.1, beta=10)
            return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        base_norm = normalize(base_image)
        secondary_norm = normalize(secondary_extracted)
        cv2.imwrite(os.path.join(OUTPUT_DIR, "base_normalized.png"), base_norm)
        cv2.imwrite(os.path.join(OUTPUT_DIR, "secondary_normalized.png"), secondary_norm)

        return mask, base_norm, secondary_norm
    except Exception as e:
        print(f"Error in preprocessing images: {e}")
        return None, None, None

In [8]:
# Step 6: Context Analysis Agent using Async/Await and Threads
async def analyze_context(base_image_rgb, secondary_image_rgb, multimedia, domain):
    loop = asyncio.get_event_loop()

    def encode_image(image):
        _, buffer = cv2.imencode('.jpg', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        b64 = base64.b64encode(buffer).decode('utf-8')
        if len(b64) > 4 * 1024 * 1024:
            raise ValueError("Image exceeds 4MB limit")
        return b64

    base_b64 = encode_image(base_image_rgb)
    secondary_b64 = encode_image(secondary_image_rgb)
    base_context = ""
    secondary_context = ""
    multimedia_context = ""
    caption = ""
    external_context = ""

    # Wrap sync LLM calls in executor for asynchronous execution
    with concurrent.futures.ThreadPoolExecutor() as executor:
        async def gemini_base():
            nonlocal base_context
            from langchain_core.messages import HumanMessage
            if gemini_llm:
                msg = HumanMessage(content=[
                    {"type": "text", "text": f"Describe the setting for a {domain} marketing image."},
                    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base_b64}"}
                ])
                result = await loop.run_in_executor(executor, gemini_llm.invoke, [msg])
                base_context = result.content
            else:
                base_context = "Default setting context"

        async def gemini_secondary():
            nonlocal secondary_context
            from langchain_core.messages import HumanMessage
            if gemini_llm:
                msg = HumanMessage(content=[
                    {"type": "text", "text": f"Identify the product features for a {domain} marketing image."},
                    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{secondary_b64}"}
                ])
                result = await loop.run_in_executor(executor, gemini_llm.invoke, [msg])
                secondary_context = result.content
            else:
                secondary_context = "Default product context"

        async def gemini_multimedia():
            nonlocal multimedia_context
            from langchain_core.messages import HumanMessage
            if multimedia and gemini_llm:
                with open(multimedia['path'], "rb") as f:
                    media_data = f.read()
                encoded_media = base64.b64encode(media_data).decode("utf-8")
                mime = "audio/mpeg" if multimedia['type'] == 'audio' else "video/mp4"
                msg = HumanMessage(content=[
                    {"type": "text", "text": f"Extract ambiance for a {domain} marketing campaign."},
                    {"type": "media", "data": encoded_media, "mime_type": mime}
                ])
                result = await loop.run_in_executor(executor, gemini_llm.invoke, [msg])
                multimedia_context = result.content
            else:
                multimedia_context = ""

        async def groq_caption():
            nonlocal caption
            response = await loop.run_in_executor(
                executor,
                lambda: groq_client.chat.completions.create(
                    model="meta-llama/llama-4-scout-17b-16e-instruct",
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {"type": "text", "text": f"Generate a JSON caption for a {domain} marketing image."},
                                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base_b64}"}},
                                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{secondary_b64}"}}
                            ]
                        }
                    ],
                    response_format={"type": "json_object"}
                )
            )
            try:
                caption = json.loads(response.choices[0].message.content).get("caption", "")
            except Exception as ex:
                caption = "Default caption"

        async def groq_external():
            nonlocal external_context
            if domain in ["travel", "automotive"]:
                response = await loop.run_in_executor(
                    executor,
                    lambda: groq_client.chat.completions.create(
                        model="meta-llama/llama-4-scout-17b-16e-instruct",
                        messages=[
                            {
                                "role": "user",
                                "content": [
                                    {"type": "text", "text": f"What's the current weather for a {domain} marketing scene?"},
                                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base_b64}"}}
                                ]
                            }
                        ],
                        tools=[{
                            "type": "function",
                            "function": {
                                "name": "get_current_weather",
                                "description": "Get the current weather in a given location",
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "location": {"type": "string"},
                                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                                    },
                                    "required": ["location"]
                                }
                            }
                        }],
                        tool_choice="auto"
                    )
                )
                if response.choices[0].message.tool_calls:
                    external_context = response.choices[0].message.tool_calls[0]["function"]["arguments"]
            else:
                external_context = ""

        await asyncio.gather(gemini_base(), gemini_secondary(), gemini_multimedia(), groq_caption(), groq_external())

    return base_context, secondary_context, multimedia_context, caption, external_context

In [9]:
# Step 7: Prompt Refinement Agent using Async Await
async def refine_prompt_async(domain, initial_prompt, base_context, secondary_context, multimedia_context, caption, external_context):
    loop = asyncio.get_event_loop()
    product = secondary_context.split(" ")[0] if secondary_context else initial_prompt.split(" ")[0]
    setting = (base_context.split(" ")[0] if base_context
               else (initial_prompt.split(" in ")[1].split(",")[0] if " in " in initial_prompt else "studio"))
    style = initial_prompt.split(",")[1].strip() if len(initial_prompt.split(",")) > 1 else "modern"
    lighting = initial_prompt.split(",")[2].strip() if len(initial_prompt.split(",")) > 2 else "natural"
    context = multimedia_context or base_context or "professional marketing aesthetic"
    caption = caption or ""
    external = external_context or ""

    from langchain.chains import LLMChain
    prompt_chain = LLMChain(llm=llm, prompt=prompt_templates.get(domain, prompt_templates["clothing"]))
    # Wrap synchronous prompt refinement in executor so it becomes async
    refined_prompt = await loop.run_in_executor(None, lambda: prompt_chain.run(
        product=product, setting=setting, style=style, lighting=lighting,
        context=context, caption=caption, external=external
    ))

    messages = [
        {"role": "user", "content": f"Refine this prompt for a high-quality {domain} marketing image: {refined_prompt}"},
        {"role": "assistant", "content": "Let’s refine further. Suggest improvements."},
        {"role": "user", "content": f"Emphasize vibrant {domain} aesthetics and detail."}
    ]

    if groq_client:
        # Wrap Groq call in executor for async execution
        response = await loop.run_in_executor(
            None,
            lambda: groq_client.chat.completions.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=messages,
                response_format={"type": "json_object"}
            )
        )
        try:
            final_prompt = json.loads(response.choices[0].message.content).get("refined_prompt", refined_prompt)
        except Exception as ex:
            final_prompt = refined_prompt
    else:
        final_prompt = refined_prompt

    print("Refined Prompt:", final_prompt)
    user_input = input("Approve or enter new prompt (press Enter to approve): ")
    if user_input.strip():
        final_prompt = user_input

    return final_prompt

In [10]:
# Step 8: Generation Agent using Async/Await and ThreadPoolExecutor
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
async def generate_composite_async(base_image, mask, prompt, domain):
    loop = asyncio.get_event_loop()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        try:
            # Try xAI generation first
            response = await loop.run_in_executor(
                executor,
                lambda: xai_client.images.generate(
                    model="grok-2-image",
                    prompt=f"{prompt}, high-quality {domain} marketing image",
                    n=2,
                    response_format="b64_json"
                )
            )
            images = []
            for idx, img_data in enumerate(response.data):
                image_b64 = img_data.b64_json
                image_data = base64.b64decode(image_b64)
                composite = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
                filename = os.path.join(OUTPUT_DIR, f"composite_image_{idx}.png")
                cv2.imwrite(filename, composite)
                images.append(composite)
            return images
        except Exception as e:
            print(f"xAI generation error: {e}. Falling back to Stable Diffusion.")
            if sd_pipeline:
                mask_img = Image.open(os.path.join(OUTPUT_DIR, "mask.png")).convert("L")
                base_pil = Image.fromarray(base_image)
                sd_pipeline.enable_model_cpu_offload()
                output = await loop.run_in_executor(
                    executor,
                    lambda: sd_pipeline(
                        prompt,
                        image=base_pil,
                        controlnet_conditioning_image=mask_img,
                        num_inference_steps=30,
                        guidance_scale=7.5
                    ).images[0]
                )
                filename = os.path.join(OUTPUT_DIR, "composite_image_0.png")
                output.save(filename)
                return [cv2.imread(filename)]
            else:
                print("Stable Diffusion unavailable, generation failed.")
                return []

NameError: name 'retry' is not defined

In [11]:
# Step 9: Enhancement Agent using Parallel Processing
def enhance_image(composite_image, domain):
    try:
        if esrgan_model:
            upscaled = esrgan_model.predict(composite_image)
            cv2.imwrite(os.path.join(OUTPUT_DIR, "upscaled_image.png"), upscaled)
        else:
            upscaled = composite_image

        if lama_model:
            h, w = upscaled.shape[:2]
            artifact_mask = np.zeros((h, w), dtype=np.uint8)
            artifact_mask[h//4:h//2, w//4:w//2] = 255
            cv2.imwrite(os.path.join(OUTPUT_DIR, "artifact_mask.png"), artifact_mask)
            inpainted = lama_model.predict(upscaled, artifact_mask)
            cv2.imwrite(os.path.join(OUTPUT_DIR, "inpainted_image.png"), inpainted)
        else:
            inpainted = upscaled

        if gemini_llm:
            _, buffer = cv2.imencode('.jpg', inpainted)
            image_b64 = base64.b64encode(buffer).decode('utf-8')
            from langchain_core.messages import HumanMessage
            style_msg = HumanMessage(content=[
                {"type": "text", "text": f"Apply a vibrant {domain} style to this image."},
                {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_b64}"}
            ])
            style_response = gemini_llm.invoke([style_msg], generation_config={"response_modalities": ["TEXT", "IMAGE"]})
            style_b64 = style_response.content[0].get("image_url", {}).get("url", "").split(",")[-1]
            if style_b64:
                style_decoded = base64.b64decode(style_b64)
                style_image = cv2.imdecode(np.frombuffer(style_decoded, np.uint8), cv2.IMREAD_COLOR)
                cv2.imwrite(os.path.join(OUTPUT_DIR, "style_image.png"), style_image)
            else:
                style_image = inpainted
        else:
            style_image = inpainted

        final_image = cv2.addWeighted(inpainted, 0.7, style_image, 0.3, 0)
        cv2.imwrite(os.path.join(OUTPUT_DIR, "final_image.png"), final_image)
        return final_image
    except Exception as e:
        print(f"Error during enhancement: {e}")
        return composite_image

In [12]:
# Step 10: Quality Evaluation Agent
def evaluate_quality(base_image, final_image):
    try:
        min_dim = (min(base_image.shape[1], final_image.shape[1]),
                   min(base_image.shape[0], final_image.shape[0]))
        base_resized = cv2.resize(base_image, min_dim)
        final_resized = cv2.resize(final_image, min_dim)
        ssim_score = ssim(base_resized, final_resized, multichannel=True)
        psnr_score = psnr(base_resized, final_resized)
        feedback = "Good quality" if ssim_score > 0.7 and psnr_score > 30 else "Low quality, consider revising prompt or style transfer."
        return ssim_score, psnr_score, feedback
    except Exception as e:
        print(f"Quality evaluation error: {e}")
        return 0, 0, "Evaluation failed"

def log_metrics(metric_name, value):
    print(f"Metric - {metric_name}: {value}")

In [13]:
# New Feature: Generate Report (Markdown)
def generate_report(final_prompt, ssim_scores, psnr_scores, feedbacks):
    report_file = os.path.join(OUTPUT_DIR, "report.md")
    with open(report_file, "w") as f:
        f.write("# Product Marketing AI Pipeline Report\n\n")
        f.write("## Final Prompt\n")
        f.write(f"{final_prompt}\n\n")
        f.write("## Quality Evaluation Metrics\n")
        for idx, (ssim_score, psnr_score, feedback) in enumerate(zip(ssim_scores, psnr_scores, feedbacks)):
            f.write(f"### Image {idx}\n")
            f.write(f"- SSIM: {ssim_score}\n")
            f.write(f"- PSNR: {psnr_score}\n")
            f.write(f"- Feedback: {feedback}\n\n")
    print(f"Report generated at: {report_file}")

In [14]:
# Step 11: Main Pipeline Execution using asyncio for parallelism
async def run_pipeline_async():
    base_image_rgb, secondary_image_rgb, multimedia, domain, initial_prompt = collect_inputs()
    if base_image_rgb is None:
        print("Input collection failed, exiting.")
        return

    mask, base_normalized, secondary_normalized = preprocess_images(base_image_rgb, secondary_image_rgb)
    if mask is None:
        print("Preprocessing failed, exiting.")
        return
    free_memory(sam)

    # Asynchronously analyze context (multiple LLM calls in parallel)
    base_context, secondary_context, multimedia_context, caption, external_context = await analyze_context(
        base_image_rgb, secondary_image_rgb, multimedia, domain
    )

    # Asynchronously refine prompt using gathered context
    final_prompt = await refine_prompt_async(domain, initial_prompt, base_context, secondary_context, multimedia_context, caption, external_context)
    print(f"Final Prompt: {final_prompt}")

    # Asynchronously generate composite images (LLM calls and image generation in parallel)
    composite_images = await generate_composite_async(base_normalized, mask, final_prompt, domain)
    if not composite_images:
        print("Image generation failed, exiting.")
        return
    free_memory(sd_pipeline)

    final_images = []
    ssim_scores = []
    psnr_scores = []
    feedbacks = []
    # Enhance images in parallel using ThreadPoolExecutor
    loop = asyncio.get_event_loop()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        tasks = []
        for idx, composite_image in enumerate(composite_images):
            task = loop.run_in_executor(executor, enhance_image, composite_image, domain)
            tasks.append(task)
        enhanced_results = await asyncio.gather(*tasks)

    for idx, final_img in enumerate(enhanced_results):
        final_images.append(final_img)
        fname = os.path.join(OUTPUT_DIR, f"final_image_{idx}.png")
        cv2.imwrite(fname, final_img)
        ssim_score, psnr_score, feedback = evaluate_quality(base_image_rgb, final_img)
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
        feedbacks.append(feedback)
        log_metrics(f"Final Image {idx} SSIM", ssim_score)
        log_metrics(f"Final Image {idx} PSNR", psnr_score)
        print(f"Image {idx} - SSIM: {ssim_score}, PSNR: {psnr_score}, Feedback: {feedback}")

    # Display images for user review
    print("Displaying final images...")
    for idx, final_img in enumerate(final_images):
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB))
        plt.title(f"Final Image {idx}")
        plt.axis("off")
        plt.show()

    selected_index_input = input("Enter the index number of the final image you prefer (press Enter to keep all): ")
    if selected_index_input.strip():
        try:
            selected_index = int(selected_index_input.strip())
            final_images = [final_images[selected_index]]
            ssim_scores = [ssim_scores[selected_index]]
            psnr_scores = [psnr_scores[selected_index]]
            feedbacks = [feedbacks[selected_index]]
            print(f"Selected Image {selected_index} as final output.")
        except Exception as e:
            print(f"Invalid input, proceeding with all images. Error: {e}")

    if input("Generate quality report? (y/n): ").lower() == 'y':
        generate_report(final_prompt, ssim_scores, psnr_scores, feedbacks)

    # Optionally, list output files available in OUTPUT_DIR
    print("Final outputs saved in the 'outputs' folder.")
    # Cleanup: Optionally remove temporary files if desired (omitted here)

def main():
    try:
        asyncio.run(run_pipeline_async())
        print("Pipeline completed successfully.")
    except Exception as e:
        print(f"Pipeline failed: {e}")

if __name__ == "__main__":
    main()

Pipeline failed: name 'asyncio' is not defined
